## Introduction to BGP Analysis using Batfish

some stuff about BGP analysis

In [2]:
# Import packages and load questions
%run startup.py

/Users/corinaminer/Batfish/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


In [3]:
# Initialize a network and snapshot
NETWORK_NAME = "bgp_network"
SNAPSHOT_NAME = "bgp_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'bgp_snapshot'

now tell them about the network and the issue: Interface utilization on core routers is badly distributed

Traffic from dept disproportionately hits core2 instead of core1 *(reason: dist2 doesn't have a route to core1)*

Introduce BGP process configuration

In [4]:
# BgpProcessConfiguration shows all configured BGP processes
bfq.bgpProcessConfiguration().answer().frame()

,Node,VRF,Router_ID,Cluster_IDs,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Neighbors,Route_Reflector,Multipath_Match_Mode
0,as2dept1,default,2.1.4.1,None,True,True,ARRIVAL_ORDER,"['2.34.101.3/32', '2.34.201.3/32']",False,EXACT_PATH
1,as2dist2,default,2.1.3.2,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.201.4/32']",False,EXACT_PATH
2,as2core1,default,2.1.2.1,['2.1.2.1'],True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
3,as1border2,default,1.2.2.2,None,True,True,ARRIVAL_ORDER,"['1.10.1.1/32', '10.13.22.3/32', '10.14.22.4/32']",False,EXACT_PATH
4,as2dist1,default,2.1.3.1,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.101.4/32']",False,EXACT_PATH
5,as3border2,default,3.2.2.2,None,True,True,ARRIVAL_ORDER,"['3.10.1.1/32', '10.13.22.1/32']",False,EXACT_PATH
6,as2core2,default,2.1.2.2,['2.1.2.2'],True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
7,as1border1,default,1.1.1.1,None,True,True,ARRIVAL_ORDER,"['1.10.1.1/32', '3.2.2.2/32', '5.6.7.8/32', '10.12.11.2/32']",False,EXACT_PATH
8,as3border1,default,3.1.1.1,None,True,True,ARRIVAL_ORDER,"['3.10.1.1/32', '10.23.21.2/32']",False,EXACT_PATH
9,as2border1,default,2.1.1.1,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '10.12.11.1/32']",False,EXACT_PATH


Can see that weirdly behaving routers as2core1 and as2dist2 do have BGP configured, so that's not the issue

Introduce BGP peer configuration

In [16]:
# BgpPeerConfiguration shows all configured BGP peers
peer = bfq.bgpPeerConfiguration().answer().frame()
peer[peer['Node'].apply(lambda x: x == 'as2core1' or x == 'as2dist2')]

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Peer_Group,Import_Policy,Export_Policy,Send_Community
12,as2core1,default,2,2.1.2.1,2,2.1.3.2,True,as2,[],[],True
15,as2core1,default,2,2.1.2.1,2,2.1.1.1,True,as2,[],[],True
16,as2dist2,default,2,2.1.3.2,2,2.1.2.1,False,as2,[],[],True
19,as2dist2,default,2,2.1.3.2,2,2.1.2.2,False,as2,[],[],True
21,as2core1,default,2,2.1.2.1,2,2.1.3.1,True,as2,[],[],True
23,as2dist2,default,2,2.34.201.3,65001,2.34.201.4,False,dept,['dept_to_as2dist'],['as2dist_to_dept'],True
24,as2core1,default,2,2.1.2.1,2,2.1.1.2,True,as2,[],[],True


No issues with peer configuration either

See what BGP sessions are established

In [17]:
# BgpSessionStatus shows whether compatible BGP sessions were established
status = bfq.bgpSessionStatus().answer().frame()
status

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
0,as1border1,default,1,as1border1:Loopback0,1.1.1.1,1,as1core1,1.10.1.1,IBGP,ESTABLISHED
1,as1border1,default,1,as1border1:GigabitEthernet1/0,10.12.11.1,2,as2border1,10.12.11.2,EBGP_SINGLEHOP,ESTABLISHED
2,as1border2,default,1,as1border2:Loopback0,1.2.2.2,1,as1core1,1.10.1.1,IBGP,ESTABLISHED
3,as1border2,default,1,as1border2:GigabitEthernet0/0,10.13.22.1,3,as3border2,10.13.22.3,EBGP_SINGLEHOP,ESTABLISHED
4,as1core1,default,1,as1core1:Loopback0,1.10.1.1,1,as1border1,1.1.1.1,IBGP,ESTABLISHED
5,as1core1,default,1,as1core1:Loopback0,1.10.1.1,1,as1border2,1.2.2.2,IBGP,ESTABLISHED
6,as2border1,default,2,as2border1:Loopback0,2.1.1.1,2,as2core1,2.1.2.1,IBGP,ESTABLISHED
7,as2border1,default,2,as2border1:Loopback0,2.1.1.1,2,as2core2,2.1.2.2,IBGP,ESTABLISHED
8,as2border1,default,2,as2border1:GigabitEthernet0/0,10.12.11.2,1,as1border1,10.12.11.1,EBGP_SINGLEHOP,ESTABLISHED
9,as2border2,default,2,as2border2:Loopback0,2.1.1.2,2,as2core1,2.1.2.1,IBGP,ESTABLISHED


will see that session is not established, so use session compatibility to drill into why

In [18]:
# BgpSessionCompatibility shows configured status of BGP sessions
compat = bfq.bgpSessionCompatibility().answer().frame()
compat

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
0,as1border1,default,1,as1border1:Loopback0,1.1.1.1,1,as1core1,1.10.1.1,IBGP,UNIQUE_MATCH
1,as1border1,default,1,None,None,666,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP
3,as1border1,default,1,as1border1:GigabitEthernet1/0,10.12.11.1,2,as2border1,10.12.11.2,EBGP_SINGLEHOP,UNIQUE_MATCH
4,as1border2,default,1,as1border2:Loopback0,1.2.2.2,1,as1core1,1.10.1.1,IBGP,UNIQUE_MATCH
5,as1border2,default,1,as1border2:GigabitEthernet0/0,10.13.22.1,3,as3border2,10.13.22.3,EBGP_SINGLEHOP,UNIQUE_MATCH
6,as1border2,default,1,as1border2:GigabitEthernet2/0,10.14.22.1,4,None,10.14.22.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
7,as1core1,default,1,as1core1:Loopback0,1.10.1.1,1,as1border1,1.1.1.1,IBGP,UNIQUE_MATCH
8,as1core1,default,1,as1core1:Loopback0,1.10.1.1,1,as1border2,1.2.2.2,IBGP,UNIQUE_MATCH
9,as2border1,default,2,as2border1:Loopback0,2.1.1.1,2,as2core1,2.1.2.1,IBGP,UNIQUE_MATCH


take opportunity to point out NO_LOCAL_IP and UNKNOWN_REMOTE issues, but skip for now with reasons why those issues can't be causing current problem

Routes between as2core1 and as2dist2 are compatible (UNIQUE_MATCH), so check routes for why the session didn't get established.

In [19]:
# Show loopback routes for as2core1
routes = bfq.routes(nodes="as2core1").answer().frame()
routes[routes['Network'].apply(lambda x: '/32' in x)]

,Node,VRF,Network,Protocol,Next_Hop_IP,Next_Hop,Admin_Distance,Metric,Tag
0,as2core1,default,2.23.12.2/32,local,AUTO/NONE(-1l),None,0,0,None
1,as2core1,default,2.23.11.2/32,local,AUTO/NONE(-1l),None,0,0,None
4,as2core1,default,2.1.3.2/32,ospf,2.23.12.3,as2dist2,110,2,None
10,as2core1,default,2.1.2.2/32,ospf,2.12.11.1,as2border1,110,3,None
11,as2core1,default,2.12.11.2/32,local,AUTO/NONE(-1l),None,0,0,None
15,as2core1,default,2.1.3.1/32,ospf,2.23.11.3,as2dist1,110,2,None
17,as2core1,default,2.1.2.1/32,connected,AUTO/NONE(-1l),None,0,0,None
18,as2core1,default,2.1.2.2/32,ospf,2.23.12.3,as2dist2,110,3,None
22,as2core1,default,2.1.2.2/32,ospf,2.23.11.3,as2dist1,110,3,None
27,as2core1,default,2.12.21.2/32,local,AUTO/NONE(-1l),None,0,0,None


Find that loopback is not advertised (missing from routes), thus solving the mystery

Join us on slack/github/etc